# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
#env = UnityEnvironment(file_name="../Tennis.app")
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
import time
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 3):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            print("DONE")
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
states[0]

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

Start off with PPO and see how it goes. Use one PPO for each agent. Split things so that when we collect trajectories, we have a shared buffer that trains the full network

In [5]:
# First create the neural network powering our agent
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):

    def __init__(self):
        super(Policy, self).__init__()
     
        self.size = 24
        self.fc1 = nn.Linear(self.size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)
        
        self.critic_fc1 = nn.Linear(self.size+4,512)
        self.critic_fc2 = nn.Linear(512,256)
        self.critic_fc3 = nn.Linear(256,1)
        self.normalDistParams = torch.ones((1,2),device=device)*0.25
        self.to(device)
        
    def forward(self, x,sampled_actions=None,other_actions=None):
        a  = torch.tensor(x,dtype=torch.float,device=device)
        a = F.relu(self.fc1(a))
        a = F.relu(self.fc2(a))
        a = F.tanh(self.fc3(a))
        
        v1 = torch.tensor(x,dtype=torch.float,device=device)
        v2 = torch.zeros(4).to(device)
        #print(v1.shape)
        if (len(v1.shape) == 2):
            v2 = torch.zeros(v1.shape[0],4).to(device)
        else:
            v2 = torch.zeros(4).to(device)
        #print(v1)
        #print(v2)
        if other_actions is not None:
            v2 = torch.tensor(other_actions,dtype=torch.float,device=device)
        if (len(v1.shape) == 2):
            v = torch.cat([v1,v2],dim=1)
        else:
            v = torch.cat([v1,v2],dim=0)
        #print(v)
        v = F.relu(self.critic_fc1(v))
        v = F.relu(self.critic_fc2(v))
        v = self.critic_fc3(v)
        
       
        #x is now the mean of a normal distribution from which we
        #sample the actual action values
        prob_dists = torch.distributions.Normal(a,self.normalDistParams)
        if sampled_actions is None:
            sampled_actions = prob_dists.sample()
        action_probabilities = prob_dists.log_prob(sampled_actions) #Prob of each individual action
        summed_action_probabilities = torch.sum(action_probabilities,dim=-1,keepdim=True)
        entropy = prob_dists.entropy().sum(-1).unsqueeze(-1)
        return sampled_actions,summed_action_probabilities,entropy,v
        


# run your own policy!
#policy=Policy().to(device)

#optimizer = optim.Adam(policy.parameters(), lr=3e-4)

In [6]:
import torch.optim as optim

In [7]:
class PPOAgent():
    
    def __init__(self):
        self.policy = Policy().to(device)
        self.optimizer = optim.Adam(self.policy.parameters(),lr=3e-4)
    
    def optimize(self, sampled_states,sampled_actions,sampled_log_probs,sampled_returns,sampled_advantages,sampled_other_actions):
        sampled_all_actions = torch.cat([sampled_actions,sampled_other_actions],dim=1)
        _, log_probs, entropy_loss, values = self.policy(sampled_states, sampled_actions,sampled_all_actions)
        #entropy_loss = torch.Tensor(np.zeros((log_probs.size(0), 1))).to(device)
       
        ratio = (log_probs - sampled_log_probs).exp()
        entropy_loss = log_probs.exp()*(sampled_log_probs)
        #print(entropy_loss.mean())
        #print(ratio)
        obj = ratio * sampled_advantages
        obj_clipped = ratio.clamp(1.0 - 0.2,
                               1.0 + 0.2) * sampled_advantages
        #print(obj_clipped)
        policy_loss = -torch.min(obj, obj_clipped).mean(0) - 0.1*entropy_loss.mean()
        #print(policy_loss)
        value_loss = 0.5 * (sampled_returns - values).pow(2).mean()
        #print(value_loss)
        #print('Policy Loss {} Value Loss {}'.format(policy_loss,value_loss))
        self.optimizer.zero_grad()
        (policy_loss + value_loss).backward()
        self.optimizer.step()

In [8]:
agent1 = PPOAgent()
agent2 = PPOAgent()

In [9]:
def gen_traj_2():
    agent1_rollout = []
    agent2_rollout = []
    rollout_length = 1000
    env_info = env.reset(train_mode=True)[brain_name]    
    states = env_info.vector_observations
    for _ in range(rollout_length):
        #states_tensor = torch.Tensor(states,device=device).cuda()
        actions1, log_probs1, _, values1 = agent1.policy(states[0])
        actions2, log_probs2, _, values2 = agent2.policy(states[1])
        
        actions = torch.cat((actions1,actions2))
        values = torch.cat((values1,values2))
        log_probs = torch.cat((log_probs1,log_probs2))
        #actions, log_probs, _, values = test_agent.policy(states)
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        terminals = np.array([1 if t else 0 for t in env_info.local_done])
        #scores += env_info.rewards
        #rollout.append([states, values.detach(), actions.detach(), log_probs.detach(), rewards, 1 - terminals])
        agent1_rollout.append([states[0],values1.detach(),actions1.detach(),log_probs1.detach(),rewards[0],1-terminals[0:1],actions2.detach()])
        agent2_rollout.append([states[1],values2.detach(),actions2.detach(),log_probs2.detach(),rewards[1],1-terminals[1:2],actions1.detach()])
        states = next_states

        dones = env_info.local_done                        # see if episode finished
        if np.any(dones):                                  # exit loop if episode finished
            break
        
    pending_value1 = agent1.policy(states[0])[-1]
    pending_value2 = agent2.policy(states[1])[-1]


    agent1_rollout.append([states[0], pending_value1, None, None, None, None,None])
    agent2_rollout.append([states[0], pending_value2, None, None, None, None,None])


    return agent1_rollout,agent2_rollout


In [10]:
def generate_processed_rollout(rollout,idx):
    #For each agent get the states, compute the rewards and also keep track of
   
    
    #Now that we have generated some trajectories
    processed_rollout = [None] * (len(rollout) - 1)
    advantages = torch.Tensor(np.zeros((1, 1))).to(device)
    returns = rollout[-1][1].detach()
    #returns = 0
    
    discount_rate = 0.99
    tau = 0.99
    for i in reversed(range(len(rollout) - 1)):
        states, value, actions, log_probs, rewards, terminals,other_actions = rollout[i]
        #terminals = torch.Tensor(terminals).unsqueeze(1).to(device)
        #rewards = torch.Tensor([rewards]).unsqueeze(1).to(device)
        rewards = torch.Tensor([rewards]).to(device)
        actions = actions
        states = torch.Tensor(states).unsqueeze(0).to(device)
        next_value = rollout[i + 1][1]
        #returns = rewards + discount_rate * terminals * returns
        returns = rewards + discount_rate * returns
        #print(returns)

        #td_error = rewards + discount_rate * terminals * next_value.detach() - value.detach()
        td_error = rewards + discount_rate * next_value.detach() - value.detach()
        #print(td_error)
        #advantages = advantages * tau * discount_rate * terminals + td_error
        advantages = advantages * tau * discount_rate + td_error
        #print(advantages)
        processed_rollout[i] = [states, actions, log_probs, returns, advantages,other_actions]
    return processed_rollout

def generate_training_data(processed_rollout,idx):
    processed_states,processed_actions,processed_log_probs,processed_returns,processed_adv,processed_other_actions = zip(*processed_rollout)
    #Convert into tensors
    processed_states = torch.cat(processed_states,dim=0)
    processed_actions = torch.cat(processed_actions,dim=0)
    processed_other_actions = torch.cat(processed_other_actions,dim=0)
    processed_log_probs = torch.cat(processed_log_probs,dim=0)
    processed_returns = torch.cat(processed_returns,dim=0)
    processed_adv = torch.cat(processed_adv,dim=0)
    return processed_states,processed_actions,processed_log_probs,processed_returns,processed_adv,processed_other_actions

    

In [11]:
def generate_processed_training_data(num_steps):
    #all_concat_train = [gen_traj_2() for i in range(num_steps)]
    all_proll1 = []
    all_proll2 = []
    for i in range(num_steps):
        roll1,roll2 = gen_traj_2()
        proll1 = generate_processed_rollout(roll1,1)
        proll2 = generate_processed_rollout(roll2,2)
        all_proll1.append(proll1)
        all_proll2.append(proll2)
        #print(len(proll2))
        
    len_final_proll1 = np.sum([len(pr) for pr in all_proll1])
    #print("FINAL LENGTH"  + str(len_final_proll1))
    final_concat_test_proll1 = [None]*(len_final_proll1)
    final_concat_test_proll2 = [None]*(len_final_proll1)
    counter = 0
    for i in range(num_steps):
        for j in range(len(all_proll1[i])):
            final_concat_test_proll1[counter] = all_proll1[i][j]
            final_concat_test_proll2[counter] = all_proll2[i][j]
            counter = counter + 1
    return final_concat_test_proll1,final_concat_test_proll2


In [ ]:
xx,yy = generate_processed_training_data(1)

In [ ]:
xx

In [12]:
num_training_steps = 2000
num_epochs_per_step = 5
all_mean_scores = []
for i in range(num_training_steps):
    #roll1,roll2 = gen_traj_2()
    #print("Rollout length " + str(len(roll1)))
    roll1,roll2 = generate_processed_training_data(40)
    step_states1,step_actions1,step_log_probs1,step_rewards1,step_advantages1,step_other_actions1 = generate_training_data(roll1,1)
    step_states2,step_actions2,step_log_probs2,step_rewards2,step_advantages2,step_other_actions2 = generate_training_data(roll2,2)
    for j in range(num_epochs_per_step):
        agent1.optimize(step_states1,step_actions1,step_log_probs1,step_rewards1,step_advantages1,step_other_actions1)
        agent2.optimize(step_states1,step_actions1,step_log_probs1,step_rewards1,step_advantages1,step_other_actions1)
   
        agent2.optimize(step_states2,step_actions2,step_log_probs2,step_rewards2,step_advantages2,step_other_actions2)
        agent1.optimize(step_states2,step_actions2,step_log_probs2,step_rewards2,step_advantages2,step_other_actions2)
    
    #Now simulate using the current policy and calculate the average score across all 20 agents
    env_info = env.reset(train_mode=True)[brain_name]    
    states = env_info.vector_observations
    current_step_scores = np.zeros(num_agents)    
    for _ in range(1000):
        actions1, log_probs1, _, values1 = agent1.policy(states[0])
        actions2, log_probs2, _, values2 = agent2.policy(states[1])
        actions = torch.cat((actions1,actions2))
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations
        current_step_scores += env_info.rewards
        states = next_states
        dones = env_info.local_done                        # see if episode finished
        if np.any(dones):                                  # exit loop if episode finished
                break

    max_score_at_this_step = np.max(current_step_scores)
    all_mean_scores.append(np.mean(max_score_at_this_step))
    if(i%50 == 0):
        print('Ep {}:Score: {}, Last 100 Rolling {} Best {} Best Last 100 {}'.format(i,max_score_at_this_step,np.mean(all_mean_scores[-100:]),np.max(all_mean_scores),np.max(all_mean_scores[-100:])))

    if(np.mean(all_mean_scores[-100:]) >= 30.0 and len(all_mean_scores) >= 100 ):
        print('Episode {}: Average Score: {}, Last 100 Rolling Average {}'.format(i,max_score_at_this_step,np.mean(all_mean_scores[-100:])))
        print("Environment is solved")
        break

c:\users\kvjos\anaconda3\envs\drlnd2\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Ep 0:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 50:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 100:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 150:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 200:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 250:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 300:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 350:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 400:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 450:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 500:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 550:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 600:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 650:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.0
Ep 700:Score: 0.0, Last 100 Rolling 0.0 Best 0.0 Best Last 100 0.

KeyboardInterrupt: 

In [ ]:
np.mean(all_mean_scores[-100:])

In [ ]:
def generate_trajectories():
    rollout = []
    rollout_length = 600
    
    
    for j in range(20):
        env_info = env.reset(train_mode=True)[brain_name]    
        states = env_info.vector_observations
        scores = np.zeros(num_agents)    
        for _ in range(rollout_length):
            #states_tensor = torch.Tensor(states,device=device).cuda()
            actions, log_probs, _, values = test_agent.policy(states)
            env_info = env.step(actions.cpu().detach().numpy())[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            terminals = np.array([1 if t else 0 for t in env_info.local_done])
            scores += env_info.rewards
            rollout.append([states, values.detach(), actions.detach(), log_probs.detach(), rewards, 1 - terminals])
            states = next_states

            dones = env_info.local_done                        # see if episode finished
            if np.any(dones):                                  # exit loop if episode finished
                break

    pending_value = policy(states)[-1]
    rollout.append([states, pending_value, None, None, None, None])
    
    #Now that we have generated some trajectories
    processed_rollout = [None] * (len(rollout) - 1)
    advantages = torch.Tensor(np.zeros((2, 1))).to(device)
    returns = pending_value.detach()
    
    discount_rate = 0.99
    tau = 0.99
    for i in reversed(range(len(rollout) - 1)):
        states, value, actions, log_probs, rewards, terminals = rollout[i]
        terminals = torch.Tensor(terminals).unsqueeze(1).to(device)
        rewards = torch.Tensor(rewards).unsqueeze(1).to(device)
        actions = actions
        states = torch.Tensor(states).to(device)
        next_value = rollout[i + 1][1]
        returns = rewards + discount_rate * terminals * returns

        td_error = rewards + discount_rate * terminals * next_value.detach() - value.detach()
        advantages = advantages * tau * discount_rate * terminals + td_error
        processed_rollout[i] = [states, actions, log_probs, returns, advantages]
    
    processed_states,processed_actions,processed_log_probs,processed_returns,processed_adv = zip(*processed_rollout)
 
    #Convert into tensors
    processed_states = torch.cat(processed_states,dim=0)
    processed_actions = torch.cat(processed_actions,dim=0)
    processed_log_probs = torch.cat(processed_log_probs,dim=0)
    processed_returns = torch.cat(processed_returns,dim=0)
    processed_adv = torch.cat(processed_adv,dim=0)
    return processed_states,processed_actions,processed_log_probs,processed_returns,processed_adv,np.mean(scores)

In [ ]:
tps,tpa,tlg,tr,tv,tms = generate_trajectories()

In [ ]:
tpa

In [ ]:
def optimize(sampled_states,sampled_actions,sampled_log_probs,sampled_returns,sampled_advantages):
    _, log_probs, entropy_loss, values = policy(sampled_states, sampled_actions)
    entropy_loss = torch.Tensor(np.zeros((log_probs.size(0), 1))).to(device)
    ratio = (log_probs - sampled_log_probs).exp()
    #print(ratio)
    obj = ratio * sampled_advantages
    obj_clipped = ratio.clamp(1.0 - 0.2,
                           1.0 + 0.2) * sampled_advantages
    #print(obj_clipped)
    policy_loss = -torch.min(obj, obj_clipped).mean(0) - 0.02 * entropy_loss.mean()
    #print(policy_loss)
    value_loss = 0.5 * (sampled_returns - values).pow(2).mean()
    #print(value_loss)

    optimizer.zero_grad()
    (policy_loss + value_loss).backward()
    #nn.utils.clip_grad_norm_(self.network.parameters(), hyperparameters['gradient_clip'])
    optimizer.step()

In [ ]:
num_training_steps = 100
num_epochs_per_step = 5
all_mean_scores = []
for i in range(num_training_steps):
    step_states,step_actions,step_log_probs,step_rewards,step_advantages,mean_score = generate_trajectories()
    for j in range(num_epochs_per_step):
        shuffled_indices = np.arange(step_states.size(0))
        np.random.shuffle(shuffled_indices)
        
        minibatch_size = 16
        num_minibatches = step_states.size(0) // minibatch_size
        for z in range(minibatch_size):
            minibatch_indices = shuffled_indices[z*minibatch_size:(z+1)*minibatch_size]
            optimize(step_states[minibatch_indices],step_actions[minibatch_indices],step_log_probs[minibatch_indices],step_rewards[minibatch_indices],step_advantages[minibatch_indices])
    
    #Now simulate using the current policy and calculate the average score across all 20 agents
    env_info = env.reset(train_mode=True)[brain_name]    
    states = env_info.vector_observations
    current_step_scores = np.zeros(num_agents)    
    for _ in range(1000):
        actions, log_probs, _, values = policy(states)
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations
        current_step_scores += env_info.rewards
        states = next_states
        dones = env_info.local_done                        # see if episode finished
        if np.any(dones):                                  # exit loop if episode finished
                break

    max_score_at_this_step = np.max(current_step_scores)
    all_mean_scores.append(np.mean(max_score_at_this_step))
    if(i%1 == 0):
        print('Episode {}: Average Score: {}, Last 100 Rolling Average {}'.format(i,max_score_at_this_step,np.mean(all_mean_scores[-100:])))

    if(np.mean(all_mean_scores[-100:]) >= 30.0 and len(all_mean_scores) >= 100 ):
        print('Episode {}: Average Score: {}, Last 100 Rolling Average {}'.format(i,max_score_at_this_step,np.mean(all_mean_scores[-100:])))
        print("Environment is solved")
        break
        
    #print('Total score (averaged over agents) this episode: {}'.format(np.mean(current_step_scores)))
    #print('Average score over the last 100 episodes: {}'.format(np.mean(all_mean_scores[-100:])))

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]
for i in range(1000):
    t_s = env_info.vector_observations                  # get the current state (for each agent)
    t_a,_,_,_ = policy(t_s)
    env_info = env.step(t_a.cpu().detach().numpy())[brain_name]
    t_n_s  = env_info.vector_observations
    t_d = env_info.local_done
    if(np.any(t_d)):
        print("DONE")
        #break;
        
    if(i % 100 == 0):
        print(t_s)
        print(t_a)
    t_s = t_n_s